wav input has been cast to required mp3 keys using the following code
  
    df.path = df.path.apply(lambda x: post_process_keys(x))

In [ ]:
!cp -r ../input/python-packages2 ./

In [ ]:
!tar xvfz ./python-packages2/jiwer.tgz
!pip install ./jiwer/jiwer-2.3.0-py3-none-any.whl -f ./ --no-index
!tar xvfz ./python-packages2/normalizer.tgz
!pip install ./normalizer/bnunicodenormalizer-0.0.24.tar.gz -f ./ --no-index
!tar xvfz ./python-packages2/pyctcdecode.tgz
!pip install ./pyctcdecode/attrs-22.1.0-py2.py3-none-any.whl -f ./ --no-index --no-deps
!pip install ./pyctcdecode/exceptiongroup-1.0.0rc9-py3-none-any.whl -f ./ --no-index --no-deps
!pip install ./pyctcdecode/hypothesis-6.54.4-py3-none-any.whl -f ./ --no-index --no-deps
!pip install ./pyctcdecode/numpy-1.21.6-cp37-cp37m-manylinux_2_12_x86_64.manylinux2010_x86_64.whl -f ./ --no-index --no-deps
!pip install ./pyctcdecode/pygtrie-2.5.0.tar.gz -f ./ --no-index --no-deps
!pip install ./pyctcdecode/sortedcontainers-2.4.0-py2.py3-none-any.whl -f ./ --no-index --no-deps
!pip install ./pyctcdecode/pyctcdecode-0.4.0-py2.py3-none-any.whl -f ./ --no-index --no-deps

!tar xvfz ./python-packages2/pypikenlm.tgz
!pip install ./pypikenlm/pypi-kenlm-0.1.20220713.tar.gz -f ./ --no-index --no-deps



In [ ]:
import os
import numpy as np
from tqdm.auto import tqdm
from glob import glob
from transformers import AutoFeatureExtractor, pipeline
import pandas as pd
import librosa
import IPython
from datasets import load_metric
from tqdm.auto import tqdm
from torch.utils.data import Dataset, DataLoader
import torch
import gc
import wave
from scipy.io import wavfile
import scipy.signal as sps
import pyctcdecode

tqdm.pandas()
import warnings
warnings.filterwarnings("ignore")



In [ ]:
# CHANGE ACCORDINGLY
BATCH_SIZE = 16
TEST_DIRECTORY = '../input/test-wav-files-dl-sprint/test_files_wav'

In [ ]:

class CFG:
    my_model_name = '../input/yellowking-dlsprint-model/YellowKing_model'
    processor_name = '../input/yellowking-dlsprint-model/YellowKing_processor'

In [ ]:
from transformers import Wav2Vec2ProcessorWithLM

processor = Wav2Vec2ProcessorWithLM.from_pretrained(CFG.processor_name)


In [ ]:


my_asrLM = pipeline("automatic-speech-recognition", model=CFG.my_model_name ,feature_extractor =processor.feature_extractor, tokenizer= processor.tokenizer,decoder=processor.decoder ,device=0)


**Following Sample Submission:**

In [ ]:
def infer(audio_path):
    speech, sr = librosa.load(audio_path, sr=processor.feature_extractor.sampling_rate)

    my_LM_prediction = my_asrLM(
                speech, chunk_length_s=112, stride_length_s=None
            )

    return my_LM_prediction['text']


In [ ]:
def batch_infer(audio_paths, batch_size=BATCH_SIZE):
    '''
    infers on a batch of audio
    args:
      audio_paths  : list of path to audio files <list of string>
    returns:
      bangla predicted texts <list of string>
    '''
    results = []
    for path in audio_paths:
        results.append(infer(path))
    
    return results

In [ ]:
from bnunicodenormalizer import Normalizer 


bnorm = Normalizer()
def normalize(sen):
    _words = [bnorm(word)['normalized']  for word in sen.split()]
    return " ".join([word for word in _words if word is not None])

def dari(sentence):
    try:
        if sentence[-1]!="।":
            sentence+="।"
    except:
        print(sentence)
    return sentence

In [ ]:
def post_process_keys(str):
    return str.replace("../input/test-wav-files-dl-sprint/test_files_wav/","").replace(".wav",".mp3")

In [ ]:
def directory_infer(audio_dir):
    '''
    infers on a directory that contains audio files
    args:
      audio_dir  : directory that contains some audio files <string>
    returns:
      a dataframe that contains 2 columns:
        * path <string>
        * sentence <string>
    '''
    # list all audio files

    audio_paths=[audio_path for audio_path in tqdm(glob(os.path.join(audio_dir,"*.*")))]
    sentences=[]
    for idx in tqdm(range(0,len(audio_paths),BATCH_SIZE)):
        batch_paths=audio_paths[idx:idx+BATCH_SIZE]
        sentences+=batch_infer(batch_paths)
        
    df= pd.DataFrame({"path":audio_paths,"sentence":sentences})
    df.sentence= df.sentence.apply(lambda x:normalize(x))
    df.sentence= df.sentence.apply(lambda x:dari(x))
    df.path = df.path.apply(lambda x: post_process_keys(x))
    
    return df 

In [ ]:
submission = directory_infer(TEST_DIRECTORY)
submission.to_csv("submission.csv", index=False)
submission.head()